## Class work wiht STT

In [1]:
!pip3 install whisper-timestamped
!pip install torch
!pip install moviepy

In [1]:
import whisper_timestamped as whisper     # For convert to text, STT (Speach to text)
import torch                              # For whisper
from moviepy.editor import VideoFileClip  # For convert MP4 to MP3
import uuid                               # For generate id to user video data
import pickle                             # Save user video data on file
import os                                 # Work with file

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [34]:
class STT:
    def __init__(self, modelType="base"):
        self.modelType = modelType
        self.batch_size = 16 # reduce if low on GPU mem
        self.divice = "cuda" if torch.cuda.is_available() else "cpu"
        self.compute_type = "float16" if torch.cuda.is_available() else "int8"
        self.model = whisper.load_model(self.modelType, device=self.divice)

    def convertMP3ToText(self, pathToMP3: str) -> (str, str):
        audio = whisper.load_audio(pathToMP3)
        result = whisper.transcribe(self.model, audio, language="ru")

        text: str = result["text"]
        word_data: list[dict] = list()
        for segment in result["segments"]:
            for word in segment["words"]:
                word_data.append({
                    "text": word["text"],
                    "startTime": word["start"] * 1000,
                    "endTime": word["end"] * 1000
                })
        return text, word_data

    def saveData(self, text: str, word_data: str) -> str:
        absPath = os.path.abspath("")
        
        # Saving data
        saveData = {
            "text": text,
            "word_data": word_data
        }
        
        # Setting save directory
        pathDict = f"{absPath}/user"
        if not os.path.exists(pathDict):
            os.makedirs(pathDict)

        # Create id user video
        _id = uuid.uuid4().hex
        while os.path.isfile(f"{pathDict}/{_id}.pickle"):
            _id = uuid.uuid4().hex
    
        # Save data in pickle
        with open(f"{pathDict}/{_id}.pickle", 'wb') as file:
            pickle.dump(saveData, file, protocol=pickle.HIGHEST_PROTOCOL)
        return _id

    def loadData(self, _id: str) -> (str, list[dict]):
        absPath = os.path.abspath("")
        pathDict = f"{absPath}/user"
        
        # Check created file 
        if not os.path.isfile(f"{pathDict}/{_id}.pickle"):
            print("File does not exist")
            return "", list()

        # Open file
        with open(f"{pathDict}/{_id}.pickle", 'rb') as file:
            data = pickle.load(file)

        return data["text"], data["word_data"]

    def convertMP3(self, pathToMP3: str) -> str:
        text, word_data = self.convertMP3ToText(pathToMP3)
        _id = self.saveData(text, word_data)
        return _id

    def convertMP4(self, pahtToMP4: str) -> str:
        absPath = os.path.abspath("")
        pathTempMP3 = f"{absPath}/temp/mp3"
        fileName = os.path.splitext(os.path.basename(pahtToMP4))[0]
        tempFilePath = f"{pathTempMP3}/{fileName}.mp3"

        # Setting temp directory
        if not os.path.exists(pathTempMP3):
            os.makedirs(pathTempMP3)
        
        # Convert MP4 to MP3
        video = VideoFileClip(pahtToMP4)
        video.audio.write_audiofile(tempFilePath, logger=None)
        
        # Convert MP3 to data text
        _id = self.convertMP3(tempFilePath)

        # Delete temp file MP3
        if os.path.exists(tempFilePath):
            os.remove(tempFilePath)
        
        return _id

In [35]:
converter = STT("base")

In [36]:
test_audio = "/Users/odner/GitHub/hack20_09_27/viral_video/test_audio/test.mp3"

In [37]:
test_video = "/Users/odner/GitHub/hack20_09_27/viral_video/test/1c6bc481dd52a9938e78e755f1e5c90e.mp4"

In [38]:
converter.convertMP3(test_audio)



  0%|                                               | 0/874 [00:00<?, ?frames/s]

100%|███████████████████████████████████| 874/874 [00:00<00:00, 1608.44frames/s]


'7c227ab1edef4fc1af5b7e972fb326b2'

In [39]:
converter.loadData("7c227ab1edef4fc1af5b7e972fb326b2")

(' Привет! Как дела? Hello! Бондиур!',
 [{'text': 'Привет!', 'startTime': 1600.0, 'endTime': 1960.0},
  {'text': 'Как', 'startTime': 2760.0, 'endTime': 3140.0},
  {'text': 'дела?', 'startTime': 3140.0, 'endTime': 3460.0},
  {'text': 'Hello!', 'startTime': 4700.0, 'endTime': 5000.0},
  {'text': 'Бондиур!', 'startTime': 7040.0, 'endTime': 7700.0}])

In [40]:
converter.convertMP4(test_video)



  0%|                                             | 0/65422 [00:00<?, ?frames/s]

  4%|█▎                              | 2784/65422 [00:02<01:00, 1040.69frames/s]

  9%|██▊                             | 5664/65422 [00:04<00:43, 1383.10frames/s]

 13%|████▏                           | 8596/65422 [00:06<00:44, 1269.74frames/s]

 17%|█████▎                         | 11224/65422 [00:09<00:45, 1187.94frames/s]

 21%|██████▌                        | 13964/65422 [00:11<00:45, 1139.97frames/s]

 24%|███████▌                        | 15580/65422 [00:15<00:56, 886.32frames/s]

 27%|████████▊                       | 17980/65422 [00:18<00:56, 834.00frames/s]

 32%|██████████▏                     | 20880/65422 [00:20<00:46, 967.54frames/s]

 36%|███████████▌                    | 23580/65422 [00:23<00:45, 925.88frames/s]

 40%|████████████▉                   | 26480/65422 [00:26<00:41, 935.03frames/s]

 45%|██████████████▎                 | 29380/65422 [00:29<00:38, 929.18frames/s]

 49%|█████████

'8f04636c19a0424c9778aee2fc6517a3'

In [41]:
converter.loadData("8f04636c19a0424c9778aee2fc6517a3")

(' Наташа, ты эту кухню видела? Это 90. Я удивляюсь, как там я тут еще вот эту, значит, уголка мягкого. Ватору, в сиденье поднимаются и там вот эти пакеты, пакеты, крышки для консервации. Чем пакет-то сразу? А картошка? С морковкой. Ты хозяйку квартиры видела? Вот она. Наташа, по-моему, она все сдует, желание. Чем по-не придется услышать это еще раз. Пойдем, неудобно. Денька получилась, пошли. Это шоу о том, как сделать ремонт без ремонта? Как своими силами преобразить квартиру без бригады рабочих, без пыли и без большого бюджесса. Использует только декорг и дизайнерский лайфхаки. Так, идем идем. Ну, лен. Прости нас, пожалуйста, мы не то, чтобы там сплеть, мы тебя в лицо можем сказать. Я все слышала. Понимаешь, ты все слышала, ну кухня тебя правда, старая блячская. Ты молодая, кухня. Ну, она нас для этого сюда и позвала. Ничего не переживая. Мы что-нибудь придумаем, что не устраивает здесь. Ну, мне не особо нравится мой стол. Сразу замечу, вот эти салфетницы, они предназначены исключит

In [33]:
del converter

## Класификация тональности

In [ ]:
import numpy as np

In [ ]:
def sentimentText(text: str) -> float:
    pass

In [ ]:
def sentimentWordData(wordData: list[str], lenWords: int = 5) -> list[bool]:
    # Data scope with sentiment for ever word
    dataScope = np.zeros(len(wordData) + lenWords*2))
    # Create zero value form start and end array
    zeroWordData = [{"text": ""} for idx in range(lenWords)]
    # New word data 
    wordData = zeroWordData + wordData + zeroWordData

    for idx in range(len(wordData) - lenWords):
        # Create text to sentiment score
        text = "".join([word["text"] for word in wordData[idx:idx+lenWords]])
        sentimentScore = sentimentText(text)

        # Add score
        dataScope[idx:idx+lenWords] += sentimentScore
    
    dataScope = dataScope[(lenWords - 1):(len(wordData) - lenWords)] / lenWords
    
    return list(dataScope)

In [49]:
!pip install vaderSentiment

In [50]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
testimonial = TextBlob("Такая вкусная еда")
print(testimonial.sentiment)

In [53]:
analyzer = SentimentIntensityAnalyzer()
sentence = "я не навижу дом"
analyzer.polarity_scores(sentence)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [57]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

In [58]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

KeyboardInterrupt: 

In [54]:
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
result = classifier("Этот фильм просто великолепен!")
print(result)


KeyboardInterrupt: 

In [59]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model="blanchefort/rubert-base-cased-sentiment")
result = classifier("Сегодня был ужасный день")
print(result)

KeyboardInterrupt: 

In [60]:
!pip install deeppavlov

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 1.2 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 1.2 MB/s eta 0:00:000:00:010:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 1.2 MB/s eta 0:00:000:00:010:00:0102
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 1.2 MB/s eta 0:00:001.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 1.2 MB/s eta 0:00:000:00:01m eta 0:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.5
    Uninstalling tqdm-4.66.5:
      Successfully uninstalled tqdm-4.66.5
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: filelock
    Found existing installation: filelock 3.13.1
    Uninstalling filelock-3.13.1:
      Successfully uninstalled filelock-3.13.1
  Attempting uninstall: scipy
    Found existing installatio

In [67]:
from deeppavlov import build_model

In [68]:
model = build_model("rusentiment_convers_bert", download=True, install=True)

Ignoring transformers: markers 'python_version < "3.8"' don't match your environment
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 1.2 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 1.2 MB/s eta 0:00:001.2 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 MB 1.2 MB/s eta 0:00:00m eta 0:00:010:00:02
  Attempting uninstall: torch
    Found existing installation: torch 2.4.0
    Uninstalling torch-2.4.0:
      Successfully uninstalled torch-2.4.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lightning 2.4.0 requires torch<4.0,>=2.1.0, but you have torch 1.13.1 which is incompatible.
pyannote-audio 3.1.1 requires torch>=2.0.0, but you have torch 1.13.1 which is incompatible.
pytorch-lightning 2.4.0 requires torch>=2.1.0, but you have torch 1.13.1 which is incompatible.
whisperx 3.1.1 requires torch>=2, but you have torch 1.13.1 which is incompatible.


2024-09-28 02:23:55.977 INFO in 'deeppavlov.core.data.utils'['utils'] at line 97: Downloading from http://files.deeppavlov.ai/v1/classifiers/rusentiment_convers_bert/rusentiment_convers_bert_torch.tar.gz to /Users/odner/.deeppavlov/models/classifiers/rusentiment_convers_bert_torch.tar.gz
100%|██████████████████████████████████████| 1.52G/1.52G [21:56<00:00, 1.15MB/s]
2024-09-28 02:45:53.163 INFO in 'deeppavlov.core.data.utils'['utils'] at line 284: Extracting /Users/odner/.deeppavlov/models/classifiers/rusentiment_convers_bert_torch.tar.gz archive into /Users/odner/.deeppavlov/models/classifiers/rusentiment_convers_bert_torch


KeyboardInterrupt: 

In [ ]:
text = ["Этот продукт мне нравится", "Ужасное качество."]
sentiment = model(text)
print(sentiment)

In [ ]:
from transformers import pipeline
model = pipeline(model="r1char9/rubert-base-cased-russian-sentiment")
model("Привет, ты мне нравишься!")

In [70]:
import numpy as np 
x = np.zeros(4)

In [71]:
x

array([0., 0., 0., 0.])

In [81]:
x = x / 3

In [82]:
x

array([4.        , 3.66666667, 3.33333333, 1.66666667])

In [83]:
!pip install 'transformers[torch]'

In [87]:
from transformers import pipeline

In [88]:
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


KeyboardInterrupt: 

In [86]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model="blanchefort/rubert-base-cased-sentiment")
result = classifier("Сегодня был ужасный день")
print(result)

KeyboardInterrupt: 